from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master("local")\
.appName("Data cleaning")\
.getOrCreate()

df = spark.read.format("csv").load('s3://mle7-data/train_numeric.csv')

In [1]:
# Works only with SparkMagic
%%info

UsageError: Cell magic `%%info` not found.


In [1]:
import os
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-pom:1.10.34,org.apache.hadoop:hadoop-aws:2.7.2 pyspark-shell'
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [12]:
filePath = "s3a://mle7-data/train_numeric.csv"
df = sqlContext.read.load(filePath,format="csv", inferSchema="true", header="true").cache()

In [13]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- L0_S0_F0: double (nullable = true)
 |-- L0_S0_F2: double (nullable = true)
 |-- L0_S0_F4: double (nullable = true)
 |-- L0_S0_F6: double (nullable = true)
 |-- L0_S0_F8: double (nullable = true)
 |-- L0_S0_F10: double (nullable = true)
 |-- L0_S0_F12: double (nullable = true)
 |-- L0_S0_F14: double (nullable = true)
 |-- L0_S0_F16: double (nullable = true)
 |-- L0_S0_F18: double (nullable = true)
 |-- L0_S0_F20: double (nullable = true)
 |-- L0_S0_F22: double (nullable = true)
 |-- L0_S1_F24: double (nullable = true)
 |-- L0_S1_F28: double (nullable = true)
 |-- L0_S2_F32: double (nullable = true)
 |-- L0_S2_F36: double (nullable = true)
 |-- L0_S2_F40: double (nullable = true)
 |-- L0_S2_F44: double (nullable = true)
 |-- L0_S2_F48: double (nullable = true)
 |-- L0_S2_F52: integer (nullable = true)
 |-- L0_S2_F56: double (nullable = true)
 |-- L0_S2_F60: double (nullable = true)
 |-- L0_S2_F64: double (nullable = true)
 |-- L0_S3_F68: doubl

In [14]:
df.count()

Py4JJavaError: An error occurred while calling o45.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8.0 failed 1 times, most recent failure: Lost task 1.0 in stage 8.0 (TID 86, localhost, executor driver): com.univocity.parsers.common.TextParsingException: java.lang.OutOfMemoryError - GC overhead limit exceeded
Parser Configuration: CsvParserSettings:
	Auto configuration enabled=true
	Autodetect column delimiter=false
	Autodetect quotes=false
	Column reordering enabled=true
	Empty value=null
	Escape unquoted values=false
	Header extraction enabled=null
	Headers=null
	Ignore leading whitespaces=false
	Ignore trailing whitespaces=false
	Input buffer size=128
	Input reading on separate thread=false
	Keep escape sequences=false
	Keep quotes=false
	Length of content displayed on error=-1
	Line separator detection enabled=false
	Maximum number of characters per column=-1
	Maximum number of columns=20480
	Normalize escaped line separators=true
	Null 

In [16]:
firstrows=df.take(5)

In [7]:
# Print with pandas
import pandas as pd

pd.DataFrame(firstrows, columns=df.columns).transpose()

,0,1,2,3,4
Id,4,6,7,9,11
L0_S0_F0,0.03,NaN,0.088,-0.036,-0.055
L0_S0_F2,-0.034,NaN,0.086,-0.064,-0.086
L0_S0_F4,-0.197,NaN,0.003,0.294,0.294
L0_S0_F6,-0.179,NaN,-0.052,0.33,0.33
L0_S0_F8,0.118,NaN,0.161,0.074,0.118
L0_S0_F10,0.116,NaN,0.025,0.161,0.025
L0_S0_F12,-0.015,NaN,-0.015,0.022,0.03
L0_S0_F14,-0.032,NaN,-0.072,0.128,0.168
L0_S0_F16,0.02,NaN,-0.225,-0.026,-0.169


In [15]:
#df.describe().toPandas().transpose()
firstrows.describe().transpose()

NameError: name 'firstrows' is not defined

In [ ]:
featureColumnNames=df.schema.names[1:969]
featureColumnNames

In [ ]:
features=df.select([c for c in df.columns if c in featureColumnNames]).take(5)

In [10]:
labels=df.select(["Response"]).take(5)
labels

[Row(Response=0),
 Row(Response=0),
 Row(Response=0),
 Row(Response=0),
 Row(Response=0)]

In [16]:
from pyspark.ml.stat import Correlation
r1 = Correlation.corr(df, "Response").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

Py4JJavaError: An error occurred while calling z:org.apache.spark.ml.stat.Correlation.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 39, localhost, executor driver): scala.MatchError: [0] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.stat.Correlation$$anonfun$1.apply(Correlation.scala:70)
	at org.apache.spark.ml.stat.Correlation$$anonfun$1.apply(Correlation.scala:70)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:393)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scala.collection.mutable.ArrayBuffer.$plus$